In [1]:
!wget -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main
!conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r

!eval "$(/usr/local/bin/conda shell.bash hook)" && conda update -y -n base -c defaults conda

import sys
sys.path.append('/usr/local/lib/python3.10/site-packages')

!eval "$(/usr/local/bin/conda shell.bash hook)" && conda create -y -n myenv python=3.10

PREFIX=/usr/local
Unpacking payload ...
entry_point.py:256: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
entry_point.py:256: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.

Installing base environment...

Preparing transaction: ...working... done
Executing transaction: ...working... done
entry_point.py:256: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best results, please verify that your PYTHONPATH only points to
    directories o

In [ ]:
%%bash
set -e

source /usr/local/etc/profile.d/conda.sh
conda init
conda activate myenv

conda install -y cudatoolkit=11.3

pip install torch==2.1.2 torchvision==0.16.2 torchaudio==2.1.2 \
  --extra-index-url https://download.pytorch.org/whl/cu113

pip install "numpy<2" cupy-cuda113

wget -O en_legal_ner_trf-any-py3-none-any.whl \
  "https://huggingface.co/opennyaiorg/en_legal_ner_trf/resolve/main/en_legal_ner_trf-any-py3-none-any.whl"

mv en_legal_ner_trf-any-py3-none-any.whl \
   en_legal_ner_trf-0.0.1-py3-none-any.whl

pip install \
  en_legal_ner_trf-0.0.1-py3-none-any.whl

pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.2.0/en_core_web_sm-3.2.0-py3-none-any.whl

git clone https://github.com/Legal-NLP-EkStep/legal_NER.git
cd legal_NER

pip install -r requirements.txt
pip install nltk utils

python -c "
import spacy
import cupy
print('spaCy GPU support:', spacy.prefer_gpu())
print('CuPy GPU device:', cupy.cuda.runtime.getDevice())
"


In [67]:
with open("/kaggle/working/legal_NER/postprocessing_utils.py", "w") as f:
    f.write("""
import re
import nltk
import spacy
import copy
import collections
import logging
import traceback
import math
import os

log_dir = "logs"
log_file = "debug_log.log"
os.makedirs(log_dir, exist_ok=True)
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(os.path.join(log_dir, log_file), mode='w', encoding='utf-8')
    ]
)
logger = logging.getLogger("postprocessing")



def get_entities(doc, labels):
    entities = []
    for ent in doc.ents:
        if ent.label_ in labels:
            entities.append(ent)
    return entities

def calculate_lev(names, threshold):
    pairs = {}
    deselect = []
    for i, name in enumerate(names):
        if i in deselect:
            continue
        pair = []

        for j in range(i + 1, len(names)):
            dis = nltk.edit_distance(name, names[j])
            if dis <= threshold:
                pair.append(j)
                deselect.append(j)
        pairs[i] = pair

    return pairs, len(pairs.keys())

def get_precedent_supras(doc, entities_pn, entities_precedents):
    text = doc.text
    ends = [ent.end_char for ent in entities_pn]
    supras = []
    for match in re.finditer(r"(\\'s\\s*case\\s*\\(supra\\)|\\s*\\(supra\\))", text):
            if match.start() in ends :
                supras.append(entities_pn[ends.index(match.start())])
            elif match.start()-1 in ends:
                supras.append(entities_pn[ends.index(match.start()-1)])

    supra_precedent_matches = {}

    for supra in supras:
        matches = []
        for i, precedent in enumerate(entities_precedents):
            if precedent.start > supra.end:
                break
            supra_text = re.sub(' +', '', supra.text)
            precedent_text = re.sub(' +', '', precedent.text)
            match = re.search(supra_text, precedent_text, re.IGNORECASE)

            if match:
                matches.append(precedent)
        if len(matches) > 0:
            supra_precedent_matches[supra] = matches[-1]

    return supra_precedent_matches,supras


def create_precedent_clusters(precedent_breakup, threshold):
    cluster_num = 0
    exclude = []
    precedent_clusters = {}
    for i, pre in enumerate(precedent_breakup.keys()):
        if i in exclude:
            continue
        pet = precedent_breakup[pre][0]
        res = precedent_breakup[pre][1]
        cit=precedent_breakup[pre][2]
        cluster = []
        cluster.append(pre)
        if pet != None and res != None:
            for j in range(i + 1, len(precedent_breakup)):
                pet_1 = list(precedent_breakup.values())[j][0]
                res_1 = list(precedent_breakup.values())[j][1]
                cit_1 = list(precedent_breakup.values())[j][2]
                if (pet_1 == None or res_1 == None) :
                    if cit_1==None:
                        exclude.append(j)
                    else:
                        if cit_1==cit:
                            exclude.append(j)
                            cluster.append(list(precedent_breakup.keys())[j])
                else:
                    dis_pet = nltk.edit_distance(pet, pet_1)
                    dis_res = nltk.edit_distance(res, res_1)
                    if dis_pet < threshold and dis_res < threshold:
                        exclude.append(j)
                        cluster.append(list(precedent_breakup.keys())[j])
            precedent_clusters[cluster_num] = cluster
            cluster_num = cluster_num + 1
        elif cit != None:
            for j in range(i + 1, len(precedent_breakup)):
                cit_1=list(precedent_breakup.values())[j][2]
                if  cit_1 !=None and  cit_1==cit:
                        exclude.append(j)
                        cluster.append(list(precedent_breakup.keys())[j])
            precedent_clusters[cluster_num] = cluster
            cluster_num = cluster_num + 1
    return precedent_clusters


def split_precedents(precedents):
    precedent_breakup = {}
    regex_vs = r'\b(?i)((v(\.|/)*s*\.*)|versus)\s+'
    regex_cit = '(\(\d+\)|\d+|\[\d+\])\s*(\(\d+\)|\d+|\[\d+\])*\s*[A-Z\.]+\s*(\(\d+\)|\d+|\[\d+\])*\s*'

    for entity in precedents:
        citation = re.search(regex_cit, entity.text)
        if citation:
            cit = citation.group()
            text = entity.text[:citation.start()]
        else:
            cit = ''
            text = entity.text
        vs = re.search(regex_vs, text)
        if vs:
            pet = (text[:vs.start()].strip())
            res = (text[vs.end():].strip())
            precedent_breakup[entity] = [pet, res, cit]
        else:
            precedent_breakup[entity] = [None, None, cit]
    return precedent_breakup


def merge_supras_precedents(precedent_supra_matches, precedent_clusters):
    counter = len(list(precedent_clusters.keys()))
    for i, s_p_match in enumerate(precedent_supra_matches.values()):
        c = 0
        for j, cluster in enumerate(precedent_clusters.values()):
            if s_p_match in cluster:
                c = 1
                cluster.append(list(precedent_supra_matches.keys())[i])
        if c == 0:
            precedent_clusters[counter] = [list(precedent_supra_matches.keys())[i], s_p_match]
            counter = counter + 1
    return precedent_clusters

def set_main_cluster(clusters):
    final_clusters = {}
    for c in clusters.keys():
        mains = max(clusters[c], key=len)
        final_clusters[mains] = clusters[c]
    return final_clusters

def precedent_coref_resol(doc):
    entities_pn = get_entities(doc, ['OTHER_PERSON', 'ORG', 'PETITIONER', 'RESPONDENT'])
    entities_precedents = get_entities(doc, ['PRECEDENT'])
    precedent_breakup = split_precedents(entities_precedents)
    precedent_clusters = create_precedent_clusters(precedent_breakup, threshold=5)
    precedent_supra_matches,supras = get_precedent_supras(doc, entities_pn, entities_precedents)
    precedent_supra_clusters = merge_supras_precedents(precedent_supra_matches, precedent_clusters)
    final_clusters = set_main_cluster(precedent_supra_clusters)
    clusters={}
    entities=[]
    for cluster in final_clusters.keys():
        if len(final_clusters[cluster])>1:
            clusters[cluster]=final_clusters[cluster]
    for entitiy in doc.ents:
        if entitiy in supras:
            entitiy.label_='PRECEDENT'
            entities.append(entitiy)
        else:
            entities.append(entitiy)
    doc.ents=entities
    return clusters


def get_roles(doc):
    other_person = []
    known_person = []
    entities = list(doc.ents)
    entities_to_remove = []
    for i, ents in enumerate(entities):
        if ents.label_ == 'OTHER_PERSON':
            entities_to_remove.append(ents)
            other_person.append(ents)
        elif ents.label_ == 'PETITIONER' or ents.label_ == 'RESPONDENT' or ents.label_ == 'JUDGE' or ents.label_ == 'WITNESS' or ents.label_ == 'LAWYER':
            known_person.append(ents)
    for ent in entities_to_remove:
        entities.remove(ent)
    return entities, other_person, known_person


def map_exact_other_person(doc):
    entities, other_person, known_person = get_roles(doc)
    other_person_text = [' '.join(oth.text.split()).lower().replace(',', '') for oth in other_person]
    ents_text = [' '.join(oth.text.split()).lower().replace(',', '') for oth in entities]
    count = 0
    other_person_found = []
    other_person_to_remove = []
    for i, other_p in enumerate(other_person):
        if other_person_text[i] in ents_text:
            labels = [entities[j].label_ for j, x in enumerate(ents_text) if other_person_text[i] == x]
            if len(set(labels)) == 1:
                count = count + 1
                other_person_to_remove.append(other_p)
                index = ents_text.index(other_person_text[i])
                other_person_found.append(other_p)
                if entities[index].label_ in ['PETITIONER', 'RESPONDENT', 'JUDGE', 'WITNESS', 'LAWYER']:
                    other_person_found[-1].label_ = entities[index].label_
    for oth in other_person_to_remove:
        other_person.remove(oth)
    return other_person, other_person_found, entities, known_person


def check_alias(names):
    names_text = [[' '.join(oth.text.split()).lower().replace(',', '').strip(), oth.label_] for oth in names]
    names_labels = []
    for i, name in enumerate(names_text):
        new_names = re.split('@|alias', name[0])
        if len(new_names) > 1:
            for n in new_names:
                names_labels.append([n.strip(), name[1], i])
        else:
            names_labels.append([name[0], name[1], i])
    return names_labels

def separate_name(names, only_first_last_name):
    aliased_cleaned_names = check_alias(names)
    separated_names = []
    for name in aliased_cleaned_names:
        separated = name[0].split(' ')
        if len(separated) > 1:
            if not only_first_last_name:
                separated_names.append([separated[-1], name[1], name[2]])
                separated_names.append([' '.join(separated[:-1]), name[1], name[2]])
        else:
            separated_names.append([separated[0], name[1], name[2]])
    return separated_names


def remove_ambiguous_names(known_person_cleaned):
    unique_known_person_cleaned = {}
    to_remove = []
    for i, el in enumerate(known_person_cleaned):
        if el[0] not in unique_known_person_cleaned.keys():
            unique_known_person_cleaned[el[0]] = [el[1]]
        else:
            unique_known_person_cleaned[el[0]].append(el[1])
    for kno in unique_known_person_cleaned.keys():
        if len(list(set(unique_known_person_cleaned[kno]))) > 1:
            to_remove.append(kno)
    known_person_left = []
    for kno in known_person_cleaned:
        if kno[0] not in to_remove:
            known_person_left.append(kno)
    known_person_cleaned_text = [other[0] for other in known_person_left]
    return known_person_cleaned_text, known_person_left

def map_name_wise_other_person(other_person_cleaned, known_person_cleaned):
    known_person_cleaned_text, known_person_left = remove_ambiguous_names(known_person_cleaned)
    c = 0
    other_person_found = []
    for i, other in enumerate(other_person_cleaned):
        if other[0] in known_person_cleaned_text:
            other_person_found.append([other[2], known_person_left[known_person_cleaned_text.index(other[0])][1]])
            c = c + 1
    return other_person_found


import logging

def other_person_coref_res(doc):
    logging.debug("Called other_person_coref_res")
    other_person, other_person_found, entities, known_person = map_exact_other_person(doc)
    logging.debug(f"other_person: {[e.text for e in other_person]}")
    logging.debug(f"other_person_found (initial): {[e.text for e in other_person_found]}")
    logging.debug(f"entities: {[e.text for e in entities]}")
    logging.debug(f"known_person: {[e.text for e in known_person]}")
    known_person_cleaned = separate_name(known_person, only_first_last_name=False)
    logging.debug(f"known_person_cleaned: {known_person_cleaned}")
    other_person_cleaned = separate_name(other_person, only_first_last_name=True)
    logging.debug(f"other_person_cleaned: {other_person_cleaned}")
    oth = map_name_wise_other_person(other_person_cleaned, known_person_cleaned)
    logging.debug(f"oth (mapping result): {oth}")
    remove = set()
    for o in oth:
        logging.debug(f"Processing mapping: {o}")
        if o[0] < len(other_person):
            entity_to_remove = other_person[o[0]]
            remove.add(entity_to_remove)
            logging.debug(f"Setting label '{o[1]}' to '{entity_to_remove.text}'")
            entity_to_remove.label_ = o[1]
            other_person_found.append(entity_to_remove)
        else:
            logging.warning(f"Invalid index {o[0]} in oth - skipping (other_person length: {len(other_person)})")
    for i in remove:
        logging.debug(f"Removing from other_person: {i.text}")
        if i in other_person:
            other_person.remove(i)
        else:
            logging.warning(f"Entity '{i.text}' not in other_person - already removed or duplicate")
    for person in other_person:
        if person not in other_person_found:
            logging.debug(f"Adding unmatched person to other_person_found: {person.text}")
            other_person_found.append(person)
    logging.debug(f"Final other_person_found before extend: {[e.text for e in other_person_found]}")
    logging.debug(f"Extending with known_person: {[e.text for e in known_person]}")
    other_person_found.extend(known_person)
    logging.debug(f"Final other_person_found: {[e.text for e in other_person_found]}")
    return other_person_found




def remove_overlapping_entities(ents, pro_sta_clusters):
    final_ents = []
    for i in ents:
        if i.label_ not in ['PETITIONER', 'RESPONDENT', 'LAWYER', 'JUDGE', 'OTHER_PERSON', 'WITNESS', 'PROVISION']:
            final_ents.append(i)
    for cluster in pro_sta_clusters:
        if cluster[0] not in final_ents:
            final_ents.append(cluster[0])
    final_ents = spacy.util.filter_spans(final_ents)
    return final_ents

def get_exact_match_pro_statute(docs):
    pro_statute = []
    pro_left = []
    total_statutes = []
    total_pros = []
    for doc in docs.sents:
        statutes = []
        pros = []
        for ents in doc.ents:
            if ents.label_ == 'STATUTE':
                statutes.append(ents)
                total_statutes.append(ents)
            elif ents.label_ == 'PROVISION':
                pros.append(ents)
                total_pros.append(ents)
        for statute in statutes:
            start = statute.start
            nearest = []
            for pro in pros:
                if pro.end <= statute.start:
                    nearest.append(statute.start - pro.end)
            if len(nearest) > 0:
                provision_ind = nearest.index(min(nearest))
                provision = pros[provision_ind]
                pros.pop(provision_ind)
                pairs = [provision, statute]
                pro_statute.append(pairs)
        if len(pros) > 0:
            pro_left.extend(pros)
    return pro_statute, pro_left, total_statutes


def separate_provision_get_pairs_statute(pro_statute):
    matching_pro_statute = []
    to_remove=[]
    sepearte_sec = r'(?i)(section(s)*|article(s)*)'
    remove_braces = r'\('
    sepearte_sub_sec = r'(?i)((sub|sub-)section(s)*|clause(s)*|annexure(s)*)'
    for pro in pro_statute:
        sub_section = re.split('of', pro[0].text)
        if len(sub_section) > 1:
            section = sub_section[1:]
        else:
            section = re.split(',|and|/|or', pro[0].text)
        for sec in section:
            match_sub_sec = re.search(sepearte_sub_sec, sec)
            if match_sub_sec:
                to_remove.append(pro)
                continue
            match_sec = re.search(sepearte_sec, sec)
            match_braces = re.search(remove_braces, sec)
            if match_braces:
                sec = sec[:match_braces.start()]
            if match_sec:
                sections = sec[match_sec.end():]
                matching_pro_statute.append([sections.strip(), pro[1]])
            else:
                matching_pro_statute.append([sec.strip(), pro[1]])
    return to_remove,matching_pro_statute


def check_validity(provision, statute):
    if 'article' in provision.text.lower():
        if 'constitution' in statute.text.lower():
            return False
        else:
            return True
    else:
        if 'constitution' in statute.text.lower():
            return True
        else:
            return False

def map_pro_statute_on_heuristics(matching_pro_left, matching_pro_statute, pro_statute, total_statutes):
    provisions_left = []
    co = 0
    for pro_left in matching_pro_left:
        provision_to_find = pro_left[0]
        sta = [i for i, v in enumerate(matching_pro_statute) if v[0] == provision_to_find]
        j = 0
        for j, statute in enumerate(sta):
            if matching_pro_statute[statute][1].start > pro_left[1].end:
                break
        if len(sta) > 0:
            if j > 0:
                sta_index = j - 1
            else:
                sta_index = 0
            statute = matching_pro_statute[sta[sta_index]]
            if pro_statute[-1][0] != pro_left[1]:
                pro_statute.append([pro_left[1], statute[1]])
                co = co + 1
            else:
                pro_statute.pop(-1)
                pro_statute.append([pro_left[1], statute[1]])
        else:
            if not total_statutes:
                logging.warning(f"Skipping pro_left '{pro_left[0]}' - no statutes available")
                continue
            i = 0
            for m, v in enumerate(total_statutes):
                if v.end > pro_left[1].end:
                    i = m
                    break
            logging.debug(f"Backtracking for pro_left '{pro_left[0]}' (provision end: {pro_left[1].end}) with initial i={i}")
            original_i = i
            while i > 0 and check_validity(pro_left[1], total_statutes[i - 1]):
                i = i - 1
                logging.debug(f"Decremented i to {i} (validity=True for statute '{total_statutes[i].text}')")
            if i > 0:
                selected_statute = total_statutes[i - 1]
                logging.debug(f"Selected prior statute '{selected_statute.text}' at index {i-1} after backtracking")
                if pro_statute[-1][0] != pro_left[1]:
                    matching_pro_statute.append([pro_left[0], selected_statute])
                    pro_statute.append([pro_left[1], selected_statute, ''])
            elif i == 0 and check_validity(pro_left[1], total_statutes[0]):
                selected_statute = total_statutes[0]
                logging.debug(f"No prior statute; using first statute '{selected_statute.text}' (validity=True)")
                if pro_statute[-1][0] != pro_left[1]:
                    matching_pro_statute.append([pro_left[0], selected_statute])
                    pro_statute.append([pro_left[1], selected_statute, ''])
            else:
                logging.warning(f"No valid prior statute found for pro_left '{pro_left[0]}' after backtracking (final i={i}, original i={original_i}) - skipping")
    return matching_pro_statute, pro_statute



def get_clusters(pro_statute, total_statute):
    custom_ents = []
    k = 0
    clusters = []
    for pro in pro_statute:
        if len(pro) > 2:
            k = k + 1
            custom_ents.append(pro)
            pro.pop(2)
        else:
            clusters.append(pro)
    for ent in custom_ents:
        clusters.append((ent[0], ent[1]))
    return clusters

def separate_provision_get_pairs_pro(pro_left):
    matching_pro_left = []
    sepearte_sec = r'(?i)(section(s)*|article(s)*)'
    remove_braces = r'\('
    sepearte_sub_sec = r'(?i)(((sub|sub-)\s*section(s)*)|clause(s)*|annexure(s)*)'
    for pro in pro_left:
        sub_section = re.split('of', pro.text)
        if len(sub_section) > 1:
            section = sub_section[1:]
        else:
            section = re.split(',|and|/|or', pro.text)
        for sec in section:
            match_sub_sec = re.search(sepearte_sub_sec, sec)
            if match_sub_sec:
                continue
            match_sec = re.search(sepearte_sec, sec)
            match_braces = re.search(remove_braces, sec)
            if match_braces:
                sec = sec[:match_braces.start()]
            if len(sec.strip()) > 0:
                if match_sec:
                    sections = sec[match_sec.end():]
                    matching_pro_left.append([sections.strip(), pro])
                else:
                    matching_pro_left.append([sec.strip(), pro])
    return matching_pro_left


def create_statute_clusters(doc,old_statute_clusters,new_statute_clusters):
    clusters = {}
    statutes = []
    not_done = []
    for ent in doc.ents:
        if ent.label_ == 'STATUTE':
            statutes.append(ent)
    for c in old_statute_clusters.keys():
        if c not in clusters.keys():
            clusters[c.text]=old_statute_clusters[c]
        else:
            clusters[c.text].extend(old_statute_clusters[c])
    for c in new_statute_clusters.keys():
        if c not in clusters.keys():
            clusters[c.text]=new_statute_clusters[c]
        else:
            clusters[c.text].extend(new_statute_clusters[c])
    for statute in statutes:
        stat = check_stat(statute.text)
        if stat == '':
            not_done.append(statute)
            continue
        if stat in clusters.keys():
            clusters[stat].append(statute)
        else:
            clusters[stat] = []
            clusters[stat].append(statute)
    return clusters


def check_stat(text):
    regex_crpc = r'(?i)\b(((criminal|cr)\.*\s*(procedure|p)\.*\s*(c|code)\.*)|(code\s*of\s*criminal\s*procedure))\s*'
    regex_ipc = r'(?i)\b((i|indian)+\.*\s*(penal|p)\.*\s*(c|code))\.*'
    regex_cons = r'(?i)\b((constitution)+\s*(of\s*india\s*)*)\b'
    regex_itact = r'(?i)\b((i\.*\s*t\.*\s*|income\s*\-*tax\s+)act\s*)\b'
    regex_mvact = r'(?i)\b((m\.*\s*v\.*\s*)|(motor\s*\-*vehicle(s)*\s+)act\s*)\b'
    regex_idact = r'(?i)\b((i\.*\s*d\.*\s*)|(industrial\s*\-*dispute(s)*\s+)act\s*)\b'
    regex_sarfaesi= r'(?i)\b((s\.*\s*a\.*\s*r\.*\s*f\.*\s*a\.*\s*e\.*\s*s\.*\s*i\.*\s*)|(securitisation\s*and\s*reconstruction\s*of\s*financial\s*assets\s*and\s*enforcement\s*of\s*security\s*interest\s+)act\s*)\b'
    match_crpc = re.search(regex_crpc, text)
    match_ipc = re.search(regex_ipc, text)
    match_cons = re.search(regex_cons, text)
    match_ita = re.search(regex_itact, text)
    match_mv = re.search(regex_mvact, text)
    match_idact = re.search(regex_idact, text)
    match_sarfaesi=re.search(regex_sarfaesi, text)
    if match_crpc:
        return 'Criminal Procedure Code'
    elif match_ipc:
        return 'Indian Penal Code'
    elif match_cons:
        return 'Constitution'
    elif match_ita:
        return 'Income Tax Act'
    elif match_mv:
        return 'Motor Vehicle Act'
    elif match_idact:
        return 'Industrial Dispute Act'
    elif match_sarfaesi:
        return 'Securitisation and Reconstruction of Financial Assets and Enforcement of Securities Interest Act'
    else:
        return ''


def remove_unidentified_statutes(doc, new_statutes):
    entities = doc.ents
    stats = []
    new_entities = []
    stats.extend(new_statutes)
    for ents in entities:
        if ents not in stats:
            new_entities.append(ents)
    return new_entities

def create_unidentified_statutes(doc):
    regex = r'\((.*?)\)'
    clusters_new_statutes = {}
    statutes = []
    for ent in doc.ents:
        if ent.label_ == 'STATUTE':
            statutes.append(ent)
    statutes_start_end = [(sta.start, sta.end) for sta in statutes]
    statutes_text = [statute.text for statute in statutes]
    for statute in statutes:
        end_char = statute.end_char
        text = doc.text[end_char:]
        match = re.search(regex, text)
        if match and match.span()[0] == 1:
            regex_act = r"\b(([A-Z][A-Za-z'']*|\d{4})(?:\s+[A-Z][a-z'']*)*\s*(a|A)ct|\s*(a|A)ct)\b"
            match1 = re.search(regex_act, match.group())
            if match1:
                stat_text = match1.group()
                if statute not in clusters_new_statutes.keys():
                    clusters_new_statutes[statute] = []
                    clusters_new_statutes[statute].append(stat_text.strip())
                else:
                    clusters_new_statutes[statute].append(stat_text.strip())
    new_statutes = []
    new_statutes_clusters = {}
    text = doc.text
    for statute in clusters_new_statutes.keys():
        for sta in clusters_new_statutes[statute]:
            ent = re.finditer(sta, text)
            stat_new = [doc.char_span(e.start(), e.end(), label="STATUTE", alignment_mode='expand') for e in ent]
            new_statutes.extend(stat_new)
            if sta not in new_statutes_clusters.keys():
                new_statutes_clusters[statute] = []
                new_statutes_clusters[statute].extend(stat_new)
            else:
                new_statutes_clusters[statute].extend(stat_new)
    discarded_statutes = []
    for sta in new_statutes:
        for s in statutes_start_end:
            if sta.start >= s[0] and sta.end <= s[1]:
                discarded_statutes.append(sta)
    old_statute_clusters={}
    for s in discarded_statutes:
        if s in new_statutes:
            new_statutes.remove(s)
    for sta in new_statutes_clusters.keys():
        for s in new_statutes_clusters[sta]:
            if s in discarded_statutes:
                new_statutes_clusters[sta].remove(s)
                if sta in old_statute_clusters.keys():
                    old_statute_clusters[sta].append(s)
                else:
                    old_statute_clusters[sta] = []
                    old_statute_clusters[sta].append(s)
    return new_statutes_clusters, new_statutes,old_statute_clusters


def add_statute_head(clusters, stat_clusters):
    new_clusters = []
    clusters_done = []
    provision_statutes = collections.namedtuple('provision_statutes', ['provision_entity', 'statute_entity', 'normalised_provision_text','normalised_statute_text'])
    for stat_cluster in stat_clusters.keys():
        acts = stat_clusters[stat_cluster]
        for i, cluster in enumerate(clusters):
            if cluster[1] in acts:
                new_clusters.append(provision_statutes(cluster[0], cluster[1], cluster[2], stat_cluster))
                clusters_done.append(cluster)
    k = 0
    for cluster in clusters:
        if cluster not in clusters_done:
            k = k + 1
            new_clusters.append(provision_statutes(cluster[0], cluster[1], cluster[2], cluster[1].text))
    return new_clusters

def separate_provision(doc, clusters):
    new_clusters = []
    for cluster in clusters:
        if len(cluster) < 2:
            logging.warning(f"Skipping invalid cluster (too short): {cluster}")
            continue
        provision = cluster[0]
        statute = cluster[1]
        if not isinstance(provision, spacy.tokens.Span) or not isinstance(statute, spacy.tokens.Span):
            logging.warning(f"Skipping cluster with invalid types: {type(provision)}, {type(statute)}")
            continue
        section = re.split(r'\b(?:,|and|or|/|&)\b', provision.text.strip())
        start = provision.start_char
        pro = provision.text
        if not section or not section[0].strip():
            logging.warning(f"Skipping cluster with empty sections: {provision.text}")
            continue
        keyword = section[0].split(' ')[0]
        if keyword and keyword[-1] == 's':
            keyword = keyword[:-1]
        combined = False
        for sec in section:
            sec_text = sec.strip()
            if len(sec_text) > 0:
                cleaned = sec_text.replace(' ', '')
                if cleaned.isalpha() or (not cleaned[0].isnumeric() and not cleaned[0].isalpha()):
                    combined = True
                    break
        if len(section) > 1 and not combined:
            for sec in section:
                sec_stripped = sec.strip()
                if not sec_stripped:
                    continue
                ind = pro.find(sec)
                if ind == -1:
                    logging.debug(f"Section '{sec}' not found in '{pro}' - skipping")
                    continue
                sect = doc.char_span(start + ind, start + ind + len(sec), "PROVISION", alignment_mode='expand')
                if sect is None:
                    logging.warning(f"Failed to create span for '{sec}' - skipping")
                    continue
                pro = pro[ind + len(sec):]
                start = start + ind + len(sec)
                if sec_stripped and not sec_stripped[0].isalpha():
                    new_clusters.append((sect, statute, keyword + ' ' + sect.text))
                else:
                    parts = sect.text.split(' ')
                    formatted = keyword + ' ' + ' '.join(parts[1:]) if len(parts) > 1 else keyword
                    new_clusters.append((sect, statute, formatted))
        else:
            new_clusters.append((cluster[0], cluster[1], cluster[0].text))
    return new_clusters

def pro_statute_coref_resol(doc):
    logging.debug("Entered pro_statute_coref_resol")
    new_statutes_clusters, new_statutes, old_statute_clusters = create_unidentified_statutes(doc)
    logging.debug(f"new_statutes_clusters: {new_statutes_clusters}")
    logging.debug(f"new_statutes: {[ent.text for ent in new_statutes]}")
    logging.debug(f"old_statute_clusters: {old_statute_clusters}")
    old_entities = list(doc.ents)
    logging.debug(f"Initial doc.ents: {[ent.text for ent in old_entities]}")
    for ent in new_statutes:
        if ent not in old_entities:
            old_entities.append(ent)
            logging.debug(f"Added new entity: {ent.text}")
    old_entities = spacy.util.filter_spans(old_entities)
    logging.debug(f"Filtered doc.ents: {[ent.text for ent in old_entities]}")
    doc.ents = old_entities
    stat_clusters = create_statute_clusters(doc, old_statute_clusters, new_statutes_clusters)
    logging.debug(f"stat_clusters: {stat_clusters}")
    pro_statute, pro_left, total_statutes = get_exact_match_pro_statute(doc)
    logging.debug(f"pro_statute: {pro_statute}")
    logging.debug(f"pro_left: {pro_left}")
    logging.debug(f"total_statutes: {total_statutes}")
    to_remove, matching_pro_statute = separate_provision_get_pairs_statute(pro_statute)
    logging.debug(f"to_remove: {to_remove}")
    logging.debug(f"matching_pro_statute (initial): {matching_pro_statute}")
    matching_pro_left = separate_provision_get_pairs_pro(pro_left)
    logging.debug(f"matching_pro_left: {matching_pro_left}")
    pro_statute = [pro for pro in pro_statute if pro not in to_remove]
    for pro in to_remove:
        logging.debug(f"Removed provision from pro_statute: {pro}")
    matching_pro_statute, pro_statute = map_pro_statute_on_heuristics(
        matching_pro_left,
        matching_pro_statute,
        pro_statute,
        total_statutes
    )
    logging.debug(f"matching_pro_statute (after heuristic): {matching_pro_statute}")
    logging.debug(f"pro_statute (after heuristic): {pro_statute}")
    clusters = get_clusters(pro_statute, total_statutes)
    logging.debug(f"clusters (before separate): {clusters}")
    valid_clusters = []
    for c in clusters:
        if len(c) < 2:
            logging.warning(f"Skipping short cluster: {c}")
            continue
        provision_text = c[0].text.strip() if isinstance(c[0], spacy.tokens.Span) else str(c[0]).strip()
        if provision_text:
            valid_clusters.append(c)
        else:
            logging.warning(f"Skipping cluster with empty provision: {c}")
    clusters = valid_clusters
    clusters = separate_provision(doc, clusters)
    logging.debug(f"clusters (after separate): {clusters}")
    new_entities = remove_unidentified_statutes(doc, new_statutes)
    logging.debug(f"new_entities after removal: {[ent.text for ent in new_entities]}")
    doc.ents = new_entities
    new_clusters = add_statute_head(clusters, stat_clusters)
    logging.debug(f"new_clusters (final): {new_clusters}")
    logging.debug(f"stat_clusters (final): {stat_clusters}")
    logging.debug("Exiting pro_statute_coref_resol")
    return new_clusters, stat_clusters



def chunk_text(text, max_length=512, overlap=50):
    import re
    tokens = re.findall(r'\\S+|\\n', text)
    chunks = []
    start = 0
    while start < len(tokens):
        end = min(start + max_length, len(tokens))
        chunk = ' '.join(tokens[start:end])
        chunks.append(chunk)
        if end == len(tokens):
            break
        start = end - overlap
    return chunks

def get_csv(doc,f_name,save_path):
    df = pd.DataFrame(columns=['file_name', 'entity', 'label', 'normalised_entities'])
    file_name=[]
    entity=[]
    label=[]
    normalised_entities=[]
    for pro_ent in doc.user_data['provision_statute_pairs']:
        file_name.append(f_name)
        entity.append(pro_ent[0])
        label.append('PROVISION')
        normalised_entities.append(pro_ent[2]+' of '+pro_ent[3])
    for pre_head in doc.user_data['precedent_clusters'].keys():
        for ent in doc.user_data['precedent_clusters'][pre_head]:
            file_name.append(f_name)
            entity.append(ent)
            label.append('PRECEDENT')
            normalised_entities.append(pre_head)
    for pre_head in doc.user_data['statute_clusters'].keys():
        for ent in doc.user_data['statute_clusters'][pre_head]:
            file_name.append(f_name)
            entity.append(ent)
            label.append('STATUTE')
            normalised_entities.append(pre_head)
    for ent in doc.ents:
        if ent not in entity:
            file_name.append(f_name)
            entity.append(ent)
            label.append(ent.label_)
            normalised_entities.append('')
    entity_text=[ent.text for ent in entity]
    df['file_name']=file_name
    df['entity']=entity_text
    df['label']=label
    df['normalised_entities']=normalised_entities
    df.to_csv(save_path)


def get_unique_precedent_count(doc):
    new_clusters={}
    clusters=doc.user_data['precedent_clusters']
    for c in clusters.keys():
        new_clusters[c]=len(clusters[c])
    return new_clusters

def get_unique_provision_count(doc):
    clusters=doc.user_data['provision_statute_pairs']
    provisions = [cluster[2]+' of '+cluster[3] for cluster in clusters]
    frequency=dict(collections.Counter(provisions))
    return frequency

def get_unique_statute_count(doc):
    clusters = doc.user_data['provision_statute_pairs']
    statutes = [cluster[3] for cluster in clusters]
    frequency = dict(collections.Counter(statutes))
    return frequency

def postprocessing(nlp_doc):
    try:
        logger.debug("Starting postprocessing.")
        precedent_clusters = None
        other_person_entites = None
        pro_sta_clusters = None
        stat_clusters = None
        all_entities = None
        try:
            logger.debug("Running precedent_coref_resol.")
            precedent_clusters = precedent_coref_resol(nlp_doc)
        except Exception as e:
            logger.error(f"Error in precedent_coref_resol: {e}")
            logger.error(traceback.format_exc())
            print(f"Error in precedent_coref_resol: {e}")
            print(traceback.format_exc())
            raise
        try:
            logger.debug("Running other_person_coref_res.")
            other_person_entites = other_person_coref_res(nlp_doc)
            logger.debug(f"Other person entities: {other_person_entites}")
        except Exception as e:
            logger.error(f"Error in other_person_coref_res: {e}")
            logger.error(traceback.format_exc())
            print(f"Error in other_person_coref_res: {e}")
            print(traceback.format_exc())
            raise
        try:
            logger.debug("Running pro_statute_coref_resol.")
            pro_sta_clusters, stat_clusters = pro_statute_coref_resol(nlp_doc)
            logger.debug(f"Provision-statute clusters: {pro_sta_clusters}")
            logger.debug(f"Statute clusters: {stat_clusters}")
        except Exception as e:
            logger.error(f"Error in pro_statute_coref_resol: {e}")
            logger.error(traceback.format_exc())
            print(f"Error in pro_statute_coref_resol: {e}")
            print(traceback.format_exc())
            raise
        try:
            logger.debug("Running remove_overlapping_entities.")
            all_entities = remove_overlapping_entities(nlp_doc.ents, pro_sta_clusters)
            logger.debug(f"Entities after removing overlaps: {all_entities}")
        except Exception as e:
            logger.error(f"Error in remove_overlapping_entities: {e}")
            logger.error(traceback.format_exc())
            print(f"Error in remove_overlapping_entities: {e}")
            print(traceback.format_exc())
            raise
        try:
            logger.debug("Extending all_entities with other_person_entites.")
            all_entities.extend(other_person_entites)
            logger.debug(f"Entities after extending: {all_entities}")
            all_entities = spacy.util.filter_spans(all_entities)
            logger.debug(f"Entities after final overlap resolution: {all_entities}")
        except Exception as e:
            logger.error(f"Error extending all_entities: {e}")
            logger.error(traceback.format_exc())
            print(f"Error extending all_entities: {e}")
            print(traceback.format_exc())
            raise
        try:
            logger.debug("Setting nlp_doc.ents and user_data.")
            nlp_doc.ents = all_entities
            nlp_doc.user_data['precedent_clusters'] = precedent_clusters
            nlp_doc.user_data['provision_statute_pairs'] = pro_sta_clusters
            nlp_doc.user_data['statute_clusters'] = stat_clusters
            logger.debug("nlp_doc user_data set successfully.")
        except Exception as e:
            logger.error(f"Error setting nlp_doc.ents or user_data: {e}")
            logger.error(traceback.format_exc())
            print(f"Error setting nlp_doc.ents or user_data: {e}")
            print(traceback.format_exc())
            raise
        logger.info("postprocessing completed successfully.")
        return nlp_doc
    except Exception as e:
        logger.critical(f"Critical error in postprocessing: {e}")
        logger.critical(traceback.format_exc())
        print(f"Critical error in postprocessing: {e}")
        print(traceback.format_exc())
        raise
""")

In [70]:
with open("legal_NER/run_inference.py", "w") as f:
    f.write('''\
import os
import logging

log_dir = "logs"
log_file = "debug_log.log"

os.makedirs(log_dir, exist_ok=True)

for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(os.path.join(log_dir, log_file), mode='w', encoding='utf-8'),
    ]
)


import spacy
spacy.require_gpu()

from pathlib import Path
from spacy import displacy
from legal_ner import (
    extract_entities_from_judgment_text,
    get_unique_precedent_count,
    get_unique_provision_count,
    get_unique_statute_count
)
import urllib.request
import json
from tqdm import tqdm


legal_nlp = spacy.load('en_legal_ner_trf')
preamble_splitting_nlp = spacy.load('en_core_web_sm')

print(legal_nlp.pipe_names)
for pipe_name in legal_nlp.pipe_names:
    component = legal_nlp.get_pipe(pipe_name)
    if hasattr(component, 'model'):
        try:
            print(f"{pipe_name} on device:", component.model.device)
        except:
            pass


input_dir = Path("/kaggle/input/indian-court-judgments-raw-text-corpus")
text_files = list(input_dir.glob("*.txt"))

output_dir = "processed_files"
os.makedirs(output_dir, exist_ok=True)


unwanted_line = "Take notes as you read a judgment using our Virtual Legal Assistant and get email alerts whenever a new judgment matches your query (Query Alert Service). Try out our Premium Member Services -- Sign up today and get free trial for one month."


error_log_path = os.path.join(log_dir, "errors.txt")

def make_json_serializable(obj):
        import collections
        if hasattr(obj, 'text'):
            return obj.text
        if isinstance(obj, tuple) and hasattr(obj, '_fields'):
            return tuple(make_json_serializable(x) for x in obj)
        if isinstance(obj, tuple):
            return tuple(make_json_serializable(x) for x in obj)
        if isinstance(obj, list):
            return [make_json_serializable(x) for x in obj]
        if isinstance(obj, dict):
            return {str(k): make_json_serializable(v) for k, v in obj.items()}
        return obj


def keys_to_string(d):
    if isinstance(d, dict):
        return {str(k): v for k, v in d.items()}
    return d

        
for i, file_path in enumerate(tqdm(text_files, desc="Processing files", unit="file")):
    tqdm.write(f"Processing file: {file_path}")
    logging.debug(f"Processing file {i+1}/{len(text_files)}: {file_path}")
    print(file_path)

    try:
        with open(file_path, "r", encoding="utf-8") as f_in:
            judgment_text = f_in.read()
        if judgment_text.startswith(unwanted_line):
            judgment_text = judgment_text[len(unwanted_line):].lstrip()
    
        run_type = 'sent'
        do_postprocess = True
        
        combined_doc = extract_entities_from_judgment_text(
            judgment_text,
            legal_nlp,
            preamble_splitting_nlp,
            run_type,
            do_postprocess
        )
        
        entities = [
            {
                "text": ent.text,
                "label": ent.label_,
                "start_char": ent.start_char,
                "end_char": ent.end_char
            }
            for ent in combined_doc.ents
        ]
        
    
        
        precedent_clusters = keys_to_string(combined_doc.user_data.get('precedent_clusters', {}))
        statute_clusters = keys_to_string(combined_doc.user_data.get('statute_clusters', {}))
        provision_statute_pairs = [
            tuple(map(str, p)) for p in combined_doc.user_data.get('provision_statute_pairs', [])
        ]
        
        precedent_counts = get_unique_precedent_count(combined_doc)
        provision_counts = get_unique_provision_count(combined_doc)
        statute_counts = get_unique_statute_count(combined_doc)
        
        
        
        
        result = {
            "entities": entities,
            "precedent_clusters": combined_doc.user_data.get('precedent_clusters', {}),
            "statute_clusters": combined_doc.user_data.get('statute_clusters', {}),
            "provision_statute_pairs": combined_doc.user_data.get('provision_statute_pairs', []),
            "precedent_counts": get_unique_precedent_count(combined_doc),
            "provision_counts": get_unique_provision_count(combined_doc),
            "statute_counts": get_unique_statute_count(combined_doc),
        }
        
        serializable_result = make_json_serializable(result)
        
    
        filename = os.path.splitext(os.path.basename(file_path))[0]
        output_path = os.path.join(output_dir, f"{filename}_ner.json")
        
        
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(serializable_result, f, ensure_ascii=False, indent=2)
    
        print(f"created json file for {output_path}")

    except Exception as e:
        error_msg = f"/n❌ Error processing file {file_path}: {str(e)}"
        tqdm.write(error_msg)
        logging.debug(error_msg)
        with open(error_log_path, "a", encoding="utf-8") as ef:
            ef.write(f"{error_msg}")

''')


In [71]:
import subprocess

python_path = "/usr/local/envs/myenv/bin/python"

process = subprocess.Popen(
    [python_path, "run_inference.py"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    universal_newlines=True,
    bufsize=1,
    cwd="legal_NER",
)

for line in process.stdout:
    print(line, end="")

process.wait()


/usr/local/envs/myenv/lib/python3.10/site-packages/torch/__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:451.)
  _C._set_default_tensor_type(t)
['transformer', 'ner']

Processing files:   0%|          | 0/400 [00:00<?, ?file/s]
                                                           

Processing files:   0%|          | 0/400 [00:00<?, ?file/s]/kaggle/working/legal_NER/data_preparation.py:261: UserWarning: [W101] Skipping Doc custom extension 'trf_data' while merging docs.
  combined_docs=spacy.tokens.Doc.from_docs(docs)
Token indices sequence length is longer than the specified maximum sequence length for this model (787 > 512). Running this sequence through the model will result in indexing errors
/kaggle/working/legal_NER/legal_ner.py:39: UserWarning: [W101] Skipping Doc custom extension 

0

In [ ]:
# with open("/kaggle/working/legal_NER/logs/debug_log.log", 'r') as f:
#     print(f.read())

In [72]:
import shutil

source_folder = "/kaggle/working/legal_NER/processed_files"
output_zip = "/kaggle/working/processed_files.zip"

shutil.make_archive(output_zip.replace(".zip", ""), 'zip', source_folder)

print("Zipping completed.")


Zipping completed.
